In [ ]:
%run ./fyers_auth.ipynb

In [ ]:
from tensorflow import keras
import sys
sys.path.append('./shared')
sys.path.append('./strategies')
from data_fetching_func import dynamicDataFetching,clean_data 
from ema_crossover import ema_crossover
import pandas as pd
import pandas_ta as ta
import datetime as dt
import numpy as np

Importing Model

In [ ]:
model = keras.models.load_model('./models/lstm_trend_1.0/')
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 first_layer (LSTM)          (None, 150)               97200     
                                                                 
 dense_layer (Dense)         (None, 1)                 151       
                                                                 
Total params: 97,351
Trainable params: 97,351
Non-trainable params: 0
_________________________________________________________________


# Trade plan


- Get trend_df and predict trend (once a day only)
- Get trend_df and predict using technical strategy
- check - 1) already in position, 
          2) money management rules
- take trade...


# Notes

- Sl and target will placed as bracket order
- while loop to check every 5min for opportunity
- run model and store trend in db
- search in db, if trend_df not found then only run model, else retrieve from db
- store all tradebook in db datewise

# Money management Rules

- 3 consecutive loss = done for the day


0 = no trade

1 = sell

2 = buy

# Trend Prediction

In [ ]:
startDate = dt.date(2020,1,20) #last working day
endDate = dt.date.today()
symbol="NSE:TATASTEEL-EQ"
resolution = "1D"
trend_df = dynamicDataFetching(fyers,startDate,endDate,symbol,resolution)
clean_data(trend_df)

In [ ]:
trend_df.shape

(686, 5)

In [ ]:
trend_df.tail()

,open,high,low,close,volume
date,,,,,
2022-10-14 05:30:00+05:30,101.85,102.80,99.9,100.10,32456805.0
2022-10-17 05:30:00+05:30,99.75,100.40,98.1,99.55,37568783.0
2022-10-18 05:30:00+05:30,100.30,101.00,99.8,100.65,30722156.0
2022-10-19 05:30:00+05:30,100.60,101.25,99.7,100.05,38764542.0
2022-10-20 05:30:00+05:30,99.50,101.20,98.8,100.90,38091315.0


In [ ]:
# Adding indicators
trend_df['RSI']=ta.rsi(trend_df.close, length=15)
trend_df['EMA_20']=ta.ema(trend_df.close, length=20)
trend_df['EMA_100']=ta.ema(trend_df.close, length=100)
trend_df['EMA_150']=ta.ema(trend_df.close, length=150)
trend_df['EMA_200']=ta.ema(trend_df.close, length=200)
trend_df['EMA_500']=ta.ema(trend_df.close, length=500)


# Dropping null values after adding indicators and features
trend_df.dropna(inplace=True)
trend_df.reset_index(inplace = True)
# trend_df.drop(['volume', 'date'], axis=1, inplace=True) # not required
trend_df.drop([ 'date'], axis=1, inplace=True) # not required

In [ ]:
trend_df = trend_df.tail(30)
trend_df.tail()


,open,high,low,close,volume,RSI,EMA_20,EMA_100,EMA_150,EMA_200,EMA_500
182,101.85,102.80,99.9,100.10,32456805.0,44.639713,101.718845,104.664759,106.781093,107.821726,93.037523
183,99.75,100.40,98.1,99.55,37568783.0,43.463089,101.512288,104.563477,106.685317,107.739421,93.063521
184,100.30,101.00,99.8,100.65,30722156.0,46.485680,101.430165,104.485982,106.605379,107.668879,93.093806
185,100.60,101.25,99.7,100.05,38764542.0,45.077278,101.298721,104.398141,106.518553,107.593069,93.121575
186,99.50,101.20,98.8,100.90,38091315.0,47.491982,101.260748,104.328871,106.444135,107.526472,93.152627


In [ ]:
trend_df.shape

(30, 11)

In [ ]:
# Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled = sc.fit_transform(trend_df)
print(data_set_scaled)

[[0.97933884 0.84063745 0.8247012  0.76344086 0.51245604 0.72822351
  0.92994796 0.94214621 1.         1.         0.        ]
 [0.86363636 0.75697211 0.83665339 0.75268817 0.41654363 0.71518736
  0.91594312 0.93682156 0.98205646 0.98017023 0.04917367]
 [0.84710744 0.80478088 0.91633466 0.89964158 0.12798515 0.88565673
  0.93770386 0.95153483 0.97729439 0.97162637 0.10553285]
 [0.94628099 0.86454183 1.         0.89964158 0.17025133 0.88565673
  0.95739215 0.96595675 0.97259538 0.96316753 0.16166705]
 [0.80578512 1.         0.86055777 1.         0.70414154 1.
  0.99871974 0.99370552 0.97679828 0.96236556 0.2226184 ]
 [1.         0.88844622 0.91633466 0.84587814 0.40742996 0.78354788
  1.         1.         0.96737032 0.94994206 0.27558452]
 [0.79752066 0.7689243  0.79681275 0.75268817 0.97724979 0.6639081
  0.97932354 0.99352973 0.94985896 0.93061037 0.32365806]
 [0.79752066 0.66533865 0.6374502  0.56630824 0.25839502 0.44892375
  0.9169467  0.96190736 0.91616303 0.89740744 0.36217738]
 

In [ ]:
data_set_scaled = np.reshape(data_set_scaled, (1, 30, 11))

In [ ]:
data_set_scaled.shape

(1, 30, 11)

In [ ]:
y_pred = model.predict(data_set_scaled)
y_pred.shape

1/1 [==============================] - 0s 356ms/step


(1, 1)

In [19]:
trend_signal = int(y_pred)
if trend_signal == 0 :
    trend = 1
elif trend_signal == 1:
    trend = 2

# Techinal Strategy

Data fetching for techinal indicator


In [13]:
startDate = dt.date(2022,10,19) #last working day
endDate = dt.date.today()
# symbol="NSE:TATASTEEL-EQ"
resolution = "5"
technical_df = dynamicDataFetching(fyers,startDate,endDate,symbol,resolution)
clean_data(technical_df)
df = technical_df.rename(columns={"open":"Open","low":"Low","high":"High","close":"Close","volume":"Volume"})

In [14]:
technical_signal = ema_crossover(technical_df)

0

Now we have both trend_signal and technical_signal, we can have final singal

In [23]:

final_signal = 0
if trend_signal == 2 and technical_signal == 2:
  final_signal = 1
elif trend_signal == 1 and technical_signal == 1:
  final_signal = -1

In [56]:
#  check if already in position
def already_in_position(positions):
    if len(positions["netPositions"]) == 0: return False
    for position in positions["netPositions"]:
        pos_symbol = position["symbol"]  # NSE:SBIN-EQ
        side =  position["side"]  # 1 for long, -1 for short, and 0 for closed

        if pos_symbol == symbol and side == final_signal:
            return True
        else:
            return False
    

In [60]:
#  check if done for the day or not
def check_three_loses(positions):
    if len(positions["netPositions"]) == 0: return False
    for position in positions["netPositions"]:
        count = 0
        if position["side"] == 0:
            if position["pl"] < 0:
                count= count+1
    return count>=3

In [61]:
positions = fyers.positions()
can_trade = (already_in_position(positions)==False) and (check_three_loses(positions)==False)

# Taking trades
if can_trade:
    stop_loss = 2*technical_df.ATR[-1]
    target = 2*stop_loss
    data = {
        "symbol":symbol,
        "qty":1,
        "type":2,
        "side":final_signal,
        "productType":"BO",
        "limitPrice":0,
        "stopPrice":0,
        "validity":"DAY",
        "disclosedQty":0,
        "offlineOrder":"False",
        "stopLoss":stop_loss,
        "takeProfit":target
    }
    fyers.place_order(data)a